In [2]:
pip install torch torchvision


  Using cached torch-2.4.1-cp38-cp38-manylinux1_x86_64.whl (797.1 MB)
  Using cached torchvision-0.19.1-cp38-cp38-manylinux1_x86_64.whl (7.0 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 11.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 15.3 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached sympy-1.13.3-py3-none-any.whl (6.2 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 8.6 MB/s eta 0:00:0000:0100:01
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
     ━━━━

In [3]:
import torch

In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [5]:
import torchvision.transforms as transforms

In [6]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Resize((227,227)),
                                transforms.Normalize([0.5], [0.5])])

In [8]:
dataset = ImageFolder("./kheer/OCT2017/train", transform=transform)

In [9]:
img, label = dataset[0]
print(img.shape, label)
img

torch.Size([3, 227, 227]) 0


tensor([[[ 0.9730,  0.9789,  0.9811,  ...,  1.0000,  1.0000,  1.0000],
         [-0.1130, -0.1356, -0.0546,  ...,  1.0000,  1.0000,  1.0000],
         [-0.7966, -0.8014, -0.7475,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [-0.9677, -0.9536, -0.9609,  ..., -0.9799, -0.9551, -0.9617],
         [-0.9548, -0.9498, -0.9739,  ..., -0.9391, -0.9560, -0.9544],
         [-0.9730, -0.9552, -0.9589,  ..., -0.9489, -0.9831, -0.9638]],

        [[ 0.9730,  0.9789,  0.9811,  ...,  1.0000,  1.0000,  1.0000],
         [-0.1130, -0.1356, -0.0546,  ...,  1.0000,  1.0000,  1.0000],
         [-0.7966, -0.8014, -0.7475,  ...,  1.0000,  1.0000,  1.0000],
         ...,
         [-0.9677, -0.9536, -0.9609,  ..., -0.9799, -0.9551, -0.9617],
         [-0.9548, -0.9498, -0.9739,  ..., -0.9391, -0.9560, -0.9544],
         [-0.9730, -0.9552, -0.9589,  ..., -0.9489, -0.9831, -0.9638]],

        [[ 0.9730,  0.9789,  0.9811,  ...,  1.0000,  1.0000,  1.0000],
         [-0.1130, -0.1356, -0.0546,  ...,  1

In [10]:
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

In [11]:
val_size = 2000
train_size = len(dataset) - val_size
train_ds,val_ds = random_split(dataset, [train_size,val_size])
len(train_ds),len(val_ds)

(81484, 2000)

In [12]:
batch_size = 64
train_loader = DataLoader(train_ds,batch_size,shuffle=True,pin_memory=True)
val_loader = DataLoader(val_ds,batch_size,pin_memory=True)

In [13]:
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class OctNet(nn.Module):
  def __init__(self):
    super(OctNet,self).__init__()
    self.conv1 = nn.Conv2d(3,32,kernel_size = 7,padding = 'same')
    self.relu = nn.ReLU()
    self.batch_norm1 = nn.BatchNorm2d(num_features = 32)
    self.max_pool = nn.MaxPool2d(kernel_size = 2,stride = 2)
    self.conv2 = nn.Conv2d(32,32,kernel_size = 7,padding = 'same')
    self.conv3 = nn.Conv2d(32,64,kernel_size = 5,padding = 'same')
    self.batch_norm2 = nn.BatchNorm2d(num_features = 64)
    self.conv4 = nn.Conv2d(64,128,kernel_size=5,padding = 'same')
    self.batch_norm3 = nn.BatchNorm2d(num_features = 128)
    self.conv5 = nn.Conv2d(128,256,kernel_size = 3,padding = 'same')
    self.batch_norm4 = nn.BatchNorm2d(num_features = 256)
    self.conv6 = nn.Conv2d(256,512,kernel_size = 3,padding = 'same')
    self.batch_norm5 = nn.BatchNorm2d(num_features = 512)
    self.avg_pool = nn.AvgPool2d(kernel_size=3)
    self.flatten = nn.Flatten()
    self.fc1 = nn.Linear(512,128)
    self.dropout = nn.Dropout(p = 0.5)
    self.fc2 = nn.Linear(128,32)
    self.fc3 = nn.Linear(32,4)
  def forward(self,c):
    x = self.conv1(c)
    x = self.relu(x)
    x = self.batch_norm1(x)
    x = self.max_pool(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.batch_norm1(x)
    x = self.max_pool(x)
    x = self.conv3(x)
    x = self.relu(x)
    x = self.batch_norm2(x)
    x = self.max_pool(x)
    x = self.conv4(x)
    x = self.relu(x)
    x = self.batch_norm3(x)
    x = self.max_pool(x)
    x = self.conv5(x)
    x = self.relu(x)
    x = self.batch_norm4(x)
    x = self.max_pool(x)
    x = self.conv6(x)
    x = self.relu(x)
    x = self.batch_norm5(x)
    x = self.max_pool(x)
    x = self.avg_pool(x)
    x = self.flatten(x)
    x = self.fc1(x)
    x = self.dropout(x)
    x = self.fc2(x)
    x = self.dropout(x)
    x = self.fc3(x)
    x = F.softmax(x,dim=1)
    return x

In [15]:
m  = OctNet()

In [16]:
b = torch.randn(32,3,227,227)
c = m(b)
print(c)

tensor([[0.2081, 0.2139, 0.3254, 0.2526],
        [0.2140, 0.3134, 0.3190, 0.1536],
        [0.1678, 0.3602, 0.1720, 0.3001],
        [0.1645, 0.3405, 0.3695, 0.1256],
        [0.2491, 0.2500, 0.1977, 0.3032],
        [0.2242, 0.3931, 0.2055, 0.1772],
        [0.2262, 0.2968, 0.3150, 0.1620],
        [0.2295, 0.3316, 0.1689, 0.2700],
        [0.2418, 0.3804, 0.1939, 0.1838],
        [0.1592, 0.3536, 0.3212, 0.1660],
        [0.1985, 0.3706, 0.2257, 0.2052],
        [0.1312, 0.5111, 0.2093, 0.1484],
        [0.1688, 0.4550, 0.1728, 0.2035],
        [0.1849, 0.1372, 0.4078, 0.2701],
        [0.2520, 0.4451, 0.1724, 0.1305],
        [0.0364, 0.6493, 0.1783, 0.1360],
        [0.0999, 0.3609, 0.4053, 0.1340],
        [0.1596, 0.4351, 0.2123, 0.1930],
        [0.2228, 0.2667, 0.1896, 0.3210],
        [0.2331, 0.3410, 0.2344, 0.1915],
        [0.2218, 0.2601, 0.3764, 0.1418],
        [0.0789, 0.2850, 0.2080, 0.4281],
        [0.1881, 0.1507, 0.2194, 0.4418],
        [0.2376, 0.3160, 0.2330, 0

In [17]:
import torch.optim as optim

In [18]:
import torch

# Select device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Enable optimized GPU usage if using a GPU
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True

# Define loss function, model, and optimizer
loss_function = torch.nn.CrossEntropyLoss()
num_epochs = 100

network = OctNet().to(device)  # Move the model to the selected device (GPU or CPU)
optimizer = torch.optim.SGD(network.parameters(), lr=0.003, momentum=0.9)

# Training loop
for epoch in range(num_epochs):
    print(f'Starting epoch {epoch + 1}')
    current_loss = 0.0
    network.train()  # Ensure model is in training mode
    correct_train, total_train = 0, 0  # Variables to accumulate training accuracy

    # Loop over batches
    for i, (inputs, targets) in enumerate(train_loader, 0):
        # Move inputs and targets to the selected device
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()  # Zero the parameter gradients
        outputs = network(inputs)  # Forward pass
        loss = loss_function(outputs, targets)  # Compute loss

        loss.backward()  # Backward pass (compute gradients)
        optimizer.step()  # Update weights

        current_loss += loss.item()

        # Training accuracy calculation
        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class
        total_train += targets.size(0)  # Update total number of examples
        correct_train += (predicted == targets).sum().item()  # Update number of correct predictions

        # Print loss and accuracy every 200 mini-batches
        if i % 200 == 199:
            train_accuracy = 100.0 * correct_train / total_train  # Training accuracy
            print(f'Loss after mini-batch {i + 1}: {current_loss / 200:.3f}')
            print(f'Training Accuracy after mini-batch {i + 1}: {train_accuracy:.2f} %')

            # Validation accuracy calculation
            network.eval()  # Switch to evaluation mode
            correct_val, total_val = 0, 0

            with torch.no_grad():  # Disable gradient calculations for validation
                for val_inputs, val_targets in val_loader:
                    val_inputs, val_targets = val_inputs.to(device), val_targets.to(device)  # Move to the selected device
                    val_outputs = network(val_inputs)
                    _, val_predicted = torch.max(val_outputs.data, 1)
                    total_val += val_targets.size(0)
                    correct_val += (val_predicted == val_targets).sum().item()

            val_accuracy = 100.0 * correct_val / total_val  # Validation accuracy
            print(f'Validation Accuracy after mini-batch {i + 1}: {val_accuracy:.2f} %')

            current_loss = 0.0  # Reset current loss for the next set of mini-batches
            correct_train, total_train = 0, 0  # Reset training accuracy metrics
            network.train()  # Switch back to training mode

print('Training complete')


/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:128: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11080). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


Starting epoch 1
Loss after mini-batch 200: 1.091
Training Accuracy after mini-batch 200: 66.34 %
Validation Accuracy after mini-batch 200: 44.10 %
Loss after mini-batch 400: 1.003
Training Accuracy after mini-batch 400: 73.64 %
Validation Accuracy after mini-batch 400: 47.30 %
Loss after mini-batch 600: 0.973
Training Accuracy after mini-batch 600: 76.94 %
Validation Accuracy after mini-batch 600: 45.95 %
Loss after mini-batch 800: 0.926
Training Accuracy after mini-batch 800: 82.04 %
Validation Accuracy after mini-batch 800: 20.25 %
Loss after mini-batch 1000: 0.911
Training Accuracy after mini-batch 1000: 83.45 %
Validation Accuracy after mini-batch 1000: 70.70 %
Loss after mini-batch 1200: 0.896
Training Accuracy after mini-batch 1200: 84.72 %
Validation Accuracy after mini-batch 1200: 53.80 %
Starting epoch 2
Loss after mini-batch 200: 0.890
Training Accuracy after mini-batch 200: 85.34 %
Validation Accuracy after mini-batch 200: 65.05 %
Loss after mini-batch 400: 0.871
Training A

In [19]:
# Save the trained model
torch.save(network.state_dict(), 'octnet_model.pth')
print('Model saved successfully')


Model saved successfully


In [20]:
# Load the saved model
network = OctNet().to(device)  # Initialize the model architecture
network.load_state_dict(torch.load('octnet_model.pth', map_location=device))  # Load the saved model state_dict
print('Model loaded successfully')


Model loaded successfully


/tmp/ipykernel_488/2901319453.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  network.load_state_dict(torch.load('octnet_model.pth', map_location=device))  # Load the sa

In [23]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Select device (GPU if available, otherwise CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the same image transformations as for the training data
test_transforms = transforms.Compose([
    transforms.Resize((227, 227)),  # Adjust to match your model's input size
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the test dataset assuming it follows ImageFolder structure
test_dataset = datasets.ImageFolder(root='./kheer/OCT2017/test', transform=test_transforms)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Load the trained model
network = OctNet().to(device)  # Move the model to the selected device (GPU or CPU)
network.load_state_dict(torch.load('octnet_model.pth'))  # Load your saved model parameters
network.eval()  # Set the model to evaluation mode

# Initialize variables to accumulate test accuracy
correct_test, total_test = 0, 0

# Testing loop (no gradient calculations required during testing)
with torch.no_grad():  # Disable gradient computation for testing
    for test_inputs, test_targets in test_loader:
        test_inputs, test_targets = test_inputs.to(device), test_targets.to(device)  # Move to the selected device

        # Forward pass to get the outputs
        test_outputs = network(test_inputs)
        _, test_predicted = torch.max(test_outputs.data, 1)  # Get the class with the highest score

        # Accumulate correct predictions and total samples
        total_test += test_targets.size(0)
        correct_test += (test_predicted == test_targets).sum().item()

# Calculate and print the test accuracy
test_accuracy = 100.0 * correct_test / total_test
print(f'Test Accuracy: {test_accuracy:.2f} %')


/tmp/ipykernel_488/2619032531.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  network.load_state_dict(torch.load('octnet_model.pth'))  # Load your saved model parameter

Test Accuracy: 96.30 %
